### I added in 2015 and 2016 data to see if El Nino makes a noticeable trend, but there was some weirdness with merging and having 13006 nan period values... will figure out what's going on in the notebook

In [6]:
import pandas as pd
import numpy as np
import random
from pathlib import Path

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
from astral import LocationInfo
from astral.sun import sun

In [7]:
root_folder = Path.cwd().parents[1]

In [24]:
#reading cleaned datasets
swell = pd.read_csv(root_folder/'data/interim/00-swell.csv')
tide = pd.read_csv(root_folder/'data/interim/00-tide.csv')
wind = pd.read_csv(root_folder/'data/interim/00-wind.csv')
period = pd.read_csv(root_folder/'data/interim/01-period.csv')

swell['UTC'] = pd.to_datetime(swell['UTC'], utc=True)
tide['UTC'] = pd.to_datetime(tide['UTC'], utc=True)
wind['UTC'] = pd.to_datetime(wind['UTC'], utc=True)
period['UTC'] = pd.to_datetime(period['UTC'], utc=True)

61368 is the number of hours in 2015 to 2021

In [9]:
print('swell hours:', swell.shape[0])
print('tide hours:', tide.shape[0])
print('wind hours:', wind.shape[0])
print('period hours:', period.shape[0])

swell hours: 59102
tide hours: 61368
wind hours: 61327
period hours: 59099


April and May have missing swell and period data in 2016, which accounts for 1464 missing hours...

In [20]:
61368 - period.shape[0]

2269

In [21]:
61368 - swell.shape[0]

2266

^ number of missing hours unaccounted for

6% not nothing....

In [12]:
period.UTC.dt.minute.value_counts()

0    59099
Name: UTC, dtype: int64

In [25]:
tw = tide.merge(wind, how='left', on='UTC')
tws = tw.merge(swell, how='left', on='UTC')
twsp = tws.merge(period, how='left', on='UTC')
twsp.isna().sum()

UTC                  0
Tide                 0
Wind Speed          63
Wind Direction     115
Height            2266
Deg               2270
Period            2269
dtype: int64

In [20]:
def checkHour(df):
    df['UTC'] = pd.to_datetime(df['UTC'], utc=True)
    onehour = timedelta(days=0, hours=1)
    baddies = []
    index = df.index.to_list()
    index.remove(0)
    for i in index:
        if (df['UTC'][i]-df['UTC'][i-1] != onehour):
            baddies.append(i)
    return len(baddies), baddies

In [21]:
checkHour(period)

(8, [4551, 10259, 27029, 30735, 35007, 36870, 42361, 50339])

^ less than 8 measurements were taken off the hour

In [36]:
nodupes = period['UTC'].value_counts() == 1
dupes = ~nodupes
period[dupes.to_list()]

,UTC,Period


^ no duplicate measurements

In [39]:
period['UTC'].dt.month.value_counts()

1     5208
8     5208
12    5208
7     5207
10    5207
9     5040
11    5040
6     4921
2     4752
3     4523
5     4465
4     4320
Name: UTC, dtype: int64

In [57]:
period.iloc[tp[tp.isna().Period.to_list()].head().UTC.index.to_list()]

,UTC,Period
4551,2015-07-09 16:23:00+00:00,20
4552,2015-07-09 17:23:00+00:00,20
4553,2015-07-09 18:23:00+00:00,17
4554,2015-07-09 19:23:00+00:00,20
4555,2015-07-09 20:23:00+00:00,20


In [76]:
period.UTC.dt.minute.value_counts()

0     37435
30    15956
23     5708
Name: UTC, dtype: int64

There are a buuuunch of measurements taken within hour intervals, but not at the top of the hour. so gotta fix that 

In [5]:
thirty_min_indicies = period[period.UTC.dt.minute==30].index
twenty3_min_indicies = period[period.UTC.dt.minute==23].index


period.loc[period.UTC.dt.minute==30, 'UTC'] = period.iloc[thirty_min_indicies].UTC-timedelta(minutes=30)
period.loc[period.UTC.dt.minute==23, 'UTC'] = period.iloc[twenty3_min_indicies].UTC-timedelta(minutes=23)

period.UTC.dt.minute.value_counts()

AttributeError: Can only use .dt accessor with datetimelike values

yew this should solve it

In [1]:
tide.merge(period, how='left', on='UTC')

NameError: name 'tide' is not defined